In [1]:
!pip -q install accelerate

In [2]:
!pip -q install peft

In [3]:
!pip -q install bitsandbytes

In [4]:
!pip -q install trl py7zr auto-gptq optimum

In [5]:
!pip -q install git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.21.2 requires transformers[sentencepiece]<4.43.0,>=4.26.0, but you have transformers 4.44.0.dev0 which is incompatible.


In [6]:
from huggingface_hub import notebook_login

In [41]:
notebook_login()

In [42]:
import torch

In [43]:
from datasets import load_dataset,Dataset

In [44]:
df=load_dataset("samsum",split="train")

In [45]:
df.shape

(14732, 3)

In [46]:
df

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [47]:
datadf=df.to_pandas()

In [48]:
datadf.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [49]:
datadf=datadf.sample(20)

In [50]:
datadf["text"]=datadf[["dialogue", "summary"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"], axis=1 )

In [51]:
datadf.head()

,id,dialogue,summary,text
4742,13811908,Violet: hi! i came across this Austin's articl...,Violet sent Claire Austin's article.,###Human: Summarize this following dialogue: V...
8870,13716431,Pat: So does anyone know when the stream is go...,Pat and Lou are waiting for The stream but Kev...,###Human: Summarize this following dialogue: P...
6554,13810214,Jane: <gif_file>\r\nJane: Whaddya think? \r\nS...,Jane is updating her Tinder profile tonight an...,###Human: Summarize this following dialogue: J...
12900,13729823,"Adam: Do u have a map of Paris?\r\nTom: Yes, W...",Tom has a map of Paris.,###Human: Summarize this following dialogue: A...
2596,13681400,"Frank: Hi, how's the family?\r\nMike: great! S...","Mike is happy, because Sam's moved out. Mike a...",###Human: Summarize this following dialogue: F...


In [52]:
datadf["text"].iloc[0]

"###Human: Summarize this following dialogue: Violet: hi! i came across this Austin's article and i thought that you might find it interesting\r\nViolet: <file_other>\r\nClaire: Hi! :) Thanks, but I've already read it. :)\r\nClaire: But thanks for thinking about me :)\n###Assistant: Violet sent Claire Austin's article."

In [53]:
datadf.head(1)

,id,dialogue,summary,text
4742,13811908,Violet: hi! i came across this Austin's articl...,Violet sent Claire Austin's article.,###Human: Summarize this following dialogue: V...


In [54]:
data=Dataset.from_pandas(datadf)

In [55]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text', '__index_level_0__'],
    num_rows: 20
})

In [56]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments

In [57]:
tokenizer=AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

In [58]:
tokenizer.eos_token

'</s>'

In [59]:
tokenizer.pad_token=tokenizer.eos_token

In [60]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [61]:
model=AutoModelForCausalLM.from_pretrained( "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                                     quantization_config=quantization_config_loading,
                                      device_map="auto")

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4664: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'mod

In [62]:
print(model)

In [63]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()

In [64]:
from peft import prepare_model_for_kbit_training

In [65]:
model = prepare_model_for_kbit_training(model)

In [66]:
from peft import LoraConfig

In [67]:
peft_config=LoraConfig(
    r=16,lora_alpha=16,lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules= ["q_proj", "v_proj"]
)

In [68]:
from peft import get_peft_model

In [69]:
model=get_peft_model(model,peft_config)

In [70]:
from trl import SFTTrainer

In [71]:
from transformers import TrainingArguments

In [99]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-samsum1",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=25,
        fp16=True,
        #push_to_hub=True
    )

In [88]:
notebook_login()

In [79]:
# model

In [101]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [102]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=25, training_loss=0.62822998046875, metrics={'train_runtime': 218.7578, 'train_samples_per_second': 0.914, 'train_steps_per_second': 0.114, 'total_flos': 59055854616576.0, 'train_loss': 0.62822998046875, 'epoch': 8.333333333333334})

In [95]:
# trainer.push_to_hub()

In [103]:
! cp -r /content/mistral-finetuned-samsum /content/drive/MyDrive/

# Inferencing

In [104]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

In [109]:
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum")

In [106]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")


In [110]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum1",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)